### here we will standardize the data for better merge performance
process stages:
1. change country names in results table to fit countries data
2. add alpha_2 country code from countries table to results table
3. change alpha_2 codes in results table to fit alpha_2 code in locations table
4. merge noaa location id to results table by city and alpha_2 code

In [ ]:
# Define the mapping dictionary
country_mapping = {
    'German DR': 'Germany',
    'Czechoslovakia': 'Czech Republic',
    'Yugoslavia': 'Serbia',
    'Serbia and Montenegro': 'Serbia',
    'Turkey': 'Türkiye',
    'Scotland' : 'United Kingdom',
    'England': 'United Kingdom',
    'Wales': 'United Kingdom',
    'Czech Republic': 'Czechia',
    'China PR': 'China',
    'Iran': 'Iran, Islamic Republic of',
    'South Korea': 'Korea, Republic of',
    'North Korea': 'Korea, Democratic People\'s Republic of',
    'Vietnam': 'Viet Nam',
    'DR Congo': 'Congo, The Democratic Republic of the',
    'Syria': 'Syrian Arab Republic',
    'Taiwan': 'Taiwan, Province of China',
    'Russia': 'Russian Federation',
    'Moldova': 'Moldova, Republic of',
    'Macedonia': 'North Macedonia',
    'Bolivia': 'Bolivia, Plurinational State of',
    'Venezuela': 'Venezuela, Bolivarian Republic of',
    'Palestine': 'Palestine, State of',
    'Tanzania': 'Tanzania, United Republic of',
    'Ivory Coast': 'Côte d\'Ivoire',
    'Micronesia': 'Micronesia, Federated States of',
    'Northern Cyprus': 'Cyprus'
}

# Update the 'results' table in the SQLite database with the new city names
for old, new in country_mapping.items():
    conn.execute('''
    UPDATE results
    SET country = ?
    WHERE country = ?
    ''', (new, old))
conn.commit()